# **Spotify Song Popularity Prediction**

---

## Table of contents

1. [Introduction](#introduction)  
2. [License](#license)  
3. [Required libraries](#required-libraries)  
4. [Problem domain](#problem-domain)  
5. [Step 1: Data Overview](#step1)  
6. [Step 2: Checking for Missing Values and Basic Cleaning](#step2)  
7. [Step 3: Exploratory Data Analysis](#step3)   
8. [Step 4: Preparing Data for Modeling](#step4)  
9. [Step 5: Splitting the Dataset](#step5) 
10. [Step 6: Data Preprocessing](#step6) 
11. [Step 7: Baseline and Advanced Modeling](#step7)  
    •	Linear Regression
	•	Random Forest Regressor
	•	XGBoost Regressor
12. [Conclusions](#conclusions)  
13. [Acknowledgements](#acknowledgements)

---

<a id="introduction"></a>
## Introduction

This report aimes at predicting the popularity of Spotify songs using a variety of numeric and categorical features. The main objective was to build and compare regression models that could accurately forecast a track’s popularity score (labeled as pop in the dataset). By carefully cleaning the data, engineering relevant features, and tuning multiple models, we sought to determine which learning algorithm could best generalize to unseen songs.

The dataset under study contained columns representing several audio and metadata attributes: tempo (bpm), loudness (dB), danceability (dnce), valence (val), energy (nrgy), acousticness (acous), speechiness (spch), release year (year), and others. We combined these numeric columns with basic categorical features (artist name and top genre) to form predictive inputs for a regression approach, using linear regression, random forest and XGBoost models. Finally, we measured performance using the Root Mean Squared Error (RMSE), which is well-suited to penalizing large prediction errors.
Throughout the analysis, we used boxplots and histograms for visualizing the distribution of features and identifying outliers, correlation analysis to understand the relationships among variables, and a learning curve to assess how each model’s performance might improve (or plateau) as the training set size increases. These steps collectively helped us refine the feature set, transform skewed data, and compare modeling approaches with greater confidence.

---

<a id="license"></a>
## License
Data from : [Kaggle Spotify dataset]( https://www.kaggle.com/competitions/cs-985-6-spotify-regression-problem-2025/overview)”. 

---

<a id="required-libraries"></a>
## Required libraries

Below are the required libraries for this project, the main libraries used here include:

- **Numpy** and **Pandas**: for data manipulation and basic data structures.
- **Matplotlib** and **Seaborn**: for plotting and visualisations.
- **Scikit-learn**: for machine learning pipelines, transformations, and modelling.
- **XGBoost**: for advanced boosting-based regression.
---


In [ ]:
#basic libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy

#plotting libraries
import matplotlib
%matplotlib inline
from matplotlib.pyplot import figure
import seaborn as sns

#importing libraries we will use for our models 

#data prep
from sklearn import preprocessing
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import TransformedTargetRegressor
#eval
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#models
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn import preprocessing
from sklearn.preprocessing import FunctionTransformer

<a id="problem-domain"></a>
## **The Problem Domain**

Spotify is a big platform for music streaming, and each song has a pop (popularity) metric. This metric captures how many plays a track receives, its skip rate, playlist additions, and more internal calculation in this dataset. The task is to train a regression model that predicts this popularity given numeric and categorical features. Among these features:

```
•    year: The year the track was released .
•    bpm: The tempo of the track in beats per minute.
•    nrgy: The energy level.
•    dnce: Danceability measure from 0–100.
•    dB: Loudness in decibels.
•    dur: Duration of the track in seconds.
•    acous: Acousticness measure (0–100).
•    spch: Speechiness measure (0–100).
•    val: Valence measure (the general positivity).
```

We have two datasets: a training set with 453 rows (after cleaning, we end up with 438) and a test set with 114 rows. Our goal is to build a reliable model that can generalize well and predict pop in the test set based on our training set.

<a id="step1"></a>
## **Step 1: Data Overview**

We began by loading two CSV files: one labeled as the training set (CS98XRegressionTrain.csv) and one as the test set (CS98XRegressionTest.csv). The training set contained 453 rows and 15 columns prior to cleaning, while the test set had 114 rows and 14 columns. An inspection with info() showed that both sets included integer, float, and object columns. The three primary object columns were title, artist, and top genre.

We also confirmed that the pop column, representing the popularity score, appeared to have valid integer data for all rows in the training set. No additional columns in the test set directly corresponded to popthat is the target variable we needed to predict.

In [ ]:
data_train=pd.read_csv("CS98XRegressionTrain.csv")
data_test= pd.read_csv('CS98XRegressionTest.csv')

# A quick glance of the data
data_columns=len(data_train.columns)
data_rows=len(data_train)
print('There are {} columns in data_train'.format(data_columns))
print('\nThere are {} rows in data_train'.format(data_rows))


data_train.head(10)

In [ ]:
data_train.info()

In [ ]:
data_train['artist'].duplicated().value_counts()

<a id="step2"></a>
## Step 2: Checking for Missing Values and Basic Cleaning

This code segment determines the total number of missing entries in both the training and test datasets, identifies which rows have null values in the top genre column, and removes those rows from the training set. The test set is checked for missing data but remains otherwise unchanged.

In [ ]:
# Missing values
n_missing=data_train.isnull().sum().sum()
print('We have {} values missing from the training data.'.format(n_missing))

n_missing_test=data_test.isnull().sum().sum()
print('We have {} values missing from the testing data'.format(n_missing_test))

missing_values=data_train.loc[data_train['top genre'].isnull() == True]
missing_values

In [ ]:
data_train.isnull().sum()

# # Drop missing rows in the training set
# data_train=data_train.dropna()

# data_test.isnull().any()



In [ ]:
data_test.isnull().sum()

<a id="step3"></a>
## Step 3: Exploratory Data Analysis

With the training data trimmed down to 438 rows, we performed an exploratory data analysis to better understand the distributions and correlations.

This section explores the dataset’s distribution and relationships between features. Histograms visualize the distribution of numerical variables, while a correlation matrix identifies features most associated with popularity (pop). Violin plots provide insights into feature distributions, and boxplots highlight outliers. Additionally, scatterplots help visualize the relationships between individual features and the target variable, aiding in feature selection and transformation decisions.

1.	**Histograms** of key features (nrgy, dnce, dB, acous) showed that some were reasonably symmetric, while others (like live and spch) were more skewed.

2.	A **correlation matrix** indicated that some variables had positive correlations with pop. Notably, dur (duration), nrgy, and dB (loudness) correlated positively with popularity, while acousticness (acous) correlated negatively. This provided early clues that well-produced, loud, and energetic songs may be more popular.

3.	**Violin plots** and **boxplots** for continuous variables further demonstrated skewed distributions in a few features, suggesting that transformations might help the model. Some outliers for dur and dnce also appeared, though not severe enough to remove.


In [ ]:
data_train.hist(bins=10,figsize=(15,20))
plt.show()
print("An inbalanced data like spch live are both right skewed")

We then prepare numeric-only data (excluding text columns like title and artist) and check correlation with the pop target:

In [ ]:
num_data_train= data_train.drop(columns=['title','artist','top genre'])
num_data_train.head(10)

In [ ]:
corr_matrix=num_data_train.corr()
corr_matrix["pop"].sort_values(ascending=False)

In [ ]:
#Correlation matrix to see any relation to the variable POP
pop_correlation=corr_matrix["pop"]
sns.heatmap(pop_correlation.to_frame(), annot=True, cmap='coolwarm', center=0, fmt=".2f", cbar=False)

In [ ]:
print("From the correlation matrix, we can see that:")
print("Positive correlation: dur, dB, nrgy, dnce, spch")
print("Negative correlation: val, live, year, acous")
print("It seems like popular songs are: less acoustic, longer duration, louder")

Visual check via a violin plot:

In [ ]:
def plot_violinplots(data, features):
    fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(20, 8))
    axes = axes.ravel()  
    
    for i, feature in enumerate(features):
        color = np.random.rand(3,)  
        sns.violinplot(x=data[feature], ax=axes[i], color=color)
        axes[i].set_title(feature)
    
    plt.tight_layout()
    plt.show()

# Example usage:
my_features = ['spch','acous','dur','val','live','dB','dnce','nrgy','bpm','year','pop']
plot_violinplots(data_train, my_features)

Function to plot some summary stats and relationships for a selected feature vs. the target:

In [ ]:
def feature_plotting(feature_name, target):

  colour = np.random.rand(3,)

  print('Summary stats for {}:\n'.format(feature_name) )
  print(data_train[feature_name].describe())
  plt.show()
  print('\nBoxplot will show us spread and highlight outliers:\n')
  sns.boxplot(x = data_train [feature_name], color = colour)
  plt.show()
  print('\nScatterplot will show us relationship between {} and {}:\n'.format(feature_name, target))
  sns.scatterplot(data=data_train,x = data_train[feature_name], y = data_train[target], color = colour)
  plt.show()

In [ ]:
feature_plotting('dur','pop')

<a id="step5"></a>
## Step 5: Feature Engineering

This step focuses on preparing the dataset for machine learning by splitting, transforming, and encoding the data appropriately.

#### Train-Test Splitting 
The function train_test() separates the dataset into training and testing sets, ensuring that transformations applied to training data do not affect the raw test set. If make_copy=True, deep copies of the datasets are created to prevent accidental modifications. The function removes the target variable (pop) from the training features while keeping it in y_train, and also drops unnecessary columns from both datasets.

#### Feature Scaling and Transformation
To ensure features are on a comparable scale, transform_features() applies either standard scaling (zero mean, unit variance) or power transformation (reducing skewness). For training data, scalers are fitted and stored in a dictionary, which is later applied to the test set to avoid data leakage.

#### Encoding Categorical Variables
Since machine learning models typically require numerical inputs, label_encoding() converts categorical features (artist, top genre) into integer values. A LabelEncoder is trained on the combined training and test data to ensure consistency in encoding.

#### Powertransformer  and Logarithm Transformations
The powertransformer are used to handle inbalance data because Features with a long tail will be replaced by their logarithm, as most models prefer features with roughly uniform or Gaussian distributions.

Functions log_transform() and inverse_transform() provide optional log transformations to stabilize variance in features or predictions.

This preprocessing step ensures that all features are scaled, encoded, and formatted properly before model training, reducing bias from feature magnitudes and improving model generalizability.

In [ ]:
#Slpliting dataset
def train_test(train_data, test_data, target, dropping_columns, make_copy=False):
    if make_copy:
        train_data = copy.deepcopy(train_data)
        test_data = copy.deepcopy(test_data)
    
    X_train = train_data.drop(target, axis=1)
    y_train = train_data[target]S
    X_test = test_data.drop(dropping_columns, axis=1)
    X_train = X_train.drop(dropping_columns, axis=1)
    
    return X_train, y_train, X_test

In [ ]:
#Applying scalling transformation
def transform_features(df, columns, transformer_name, transf_dict):
  #function for scaling or normalising specified features

  if transf_dict is None:

    if transformer_name == 'Standard':
      transf = preprocessing.StandardScaler()
  
    elif transformer_name == 'PowerTransformer':
      transf = preprocessing.PowerTransformer()
  
    #make a dictionary of the fitted transformer so that we can use training scales for testing data

    transf_dict = {}
    
    for col in columns:
      col_transf = copy.deepcopy(transf)
      col_transf = col_transf.fit(df[[col]])
      df[col] = col_transf.transform(df[[col]])
      transf_dict[col] = col_transf
	
  else:
    for col in columns:
      transf = transf_dict.get(col)
      df[col] = transf.transform(df[[col]])
    
    return df

  return df, transf_dict #returning dictionary and df with scaled features

In [ ]:
#Label encoding for categorical features

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

def label_encoding(X_train, X_test, column):
   
    imputer = SimpleImputer(strategy='most_frequent')  
    X_train[column] = imputer.fit_transform(X_train[[column]]).ravel()
    X_test[column] = imputer.transform(X_test[[column]]).ravel()
    
    train_len = len(X_train)
    label_encoder = LabelEncoder()
    full_data = pd.concat([X_train[column], X_test[column]], axis=0, ignore_index=True)
    full_data_enc = label_encoder.fit_transform(full_data.values)
    
    X_train_enc = full_data_enc[:train_len]
    X_test_enc = full_data_enc[train_len:]
    
    return X_train_enc, X_test_enc, label_encoder

In [ ]:
#Logarithmic and exponential transformation
def log_transform(x):
    return np.log(x)

def inverse_transform(x):
    return np.exp(x)

In [ ]:
#Crossvalidation Scores
def display_scores(scores, negative_result=False):
    if negative_result:
        scores = np.array([abs(x) for x in scores])
    
    print("Scores:")
    for i, score in enumerate(scores):
        print('Fold {} - {:.3f}'.format(i, score))
    
    print("\nMean: {:.3f}".format(scores.mean()))
    print("Standard deviation: {:.3f}".format(scores.std()))
    print('\n')

In [ ]:
#Function to compute and visualize the learning curve
def learning_curve(model, X, y, xgb=False):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    train_errors, val_errors = [], []
    
    if xgb:
        y_train = y_train.values.ravel()
    
    for m in range(1, X_train.shape[0]):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

In [ ]:
#Function learning curve
def plot_learning_curve(model, X, y, xgb=False):
    plt.figure(figsize=(6, 4), dpi=120)
    learning_curve(model, X, y, xgb=xgb)
    plt.ylim(top=20)
    plt.legend()
    plt.xlabel('Training set size')
    plt.ylabel('RMSE')

In [ ]:
#Prepare dataset
target = 'pop'
dropping_columns = ['Id', 'title']
X_train, y_train, X_test = train_test(data_train, data_test, target, dropping_columns, make_copy=True)

In [ ]:
#Dataset info
print('There are {} rows in X_train'.format(len(X_train)))
print('There are {} columns in X_train'.format(len(X_train.columns)))
print('There are {} rows in X_test'.format(len(X_test)))
X_train

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

<a id="step6"></a>
## Step 6: Data Preprocessing

To enhance feature representation, two new variables were engineered: new1, a weighted combination of loudness (dB), duration (dur), and danceability (dnce), and new2, a product of danceability (dnce) and energy (nrgy). Categorical variables, specifically artist and top genre, were label-encoded to convert them into numerical format.

For continuous variables, a Power Transformer was applied to mitigate skewness and normalize feature distributions, followed by Standard Scaling to ensure zero mean and unit variance. These transformations were fitted on the training data and subsequently applied to the test set to prevent data leakage.

In [ ]:
# data prep
X_train["new1"]=0.4*X_train["dB"]+0.4*X_train["dur"]+0.2*X_train["dnce"]#try 1
X_test["new1"]=0.4*X_test["dB"]+0.4*X_test["dur"]+0.2*X_test["dnce"]#try 1

X_train["new2"]=X_train["dnce"]*X_train["nrgy"] # try 2
X_test["new2"]=X_test["dnce"]*X_test["nrgy"] # try 2

X_train['nrgy_dB_ratio'] = X_train['nrgy'] / (X_train['dB'] + 1e-6) #try3
X_test['nrgy_dB_ratio'] = X_test['nrgy'] / (X_test['dB'] + 1e-6) #try3

X_train['dur_nrgy'] = X_train['dur'] * X_train['nrgy'] #try4
X_test['dur_nrgy'] = X_test['dur'] * X_test['nrgy'] #try4


#label encod -artist
column1='artist'
X_train_enc_artist,X_test_enc_artist,label_encoder_art=label_encoding(X_train,X_test,column1)
X_train['artist']=X_train_enc_artist
X_test['artist']=X_test_enc_artist

#label encod -top genre
column2='top genre'
X_train_enc_genre,X_test_enc_genre,label_encoder_genre=label_encoding(X_train,X_test,column2)
X_train['top genre']=X_train_enc_genre
X_test['top genre']=X_test_enc_genre

#scaling and transforming conitnuous data
cont_data = ['year', 'bpm','nrgy',	'dnce',	'dB',	'live',	'val', 'dur',	'acous', 'spch','new1','new2','nrgy_dB_ratio','dur_nrgy','log_nrgy']

#Power transforming continuous features
X_train, powertf_dict = transform_features(X_train, cont_data, 'PowerTransformer', transf_dict = None)
X_test = transform_features(X_test, cont_data, 'PowerTransformer', transf_dict = powertf_dict)

#Standardizing continuous features
X_train, standard_scalers = transform_features(X_train, cont_data, 'Standard', transf_dict = None)
X_test = transform_features(X_test, cont_data, 'Standard', transf_dict = standard_scalers)


In [ ]:
X_train.hist(bins=10,figsize=(15,20))
plt.show()

<a id="step7"></a>
## Step 7: Baseline and Advanced Modeling
We evaluated three primary regression models: **Linear Regression**, **Random Forest Regressor**, and **XGBoost Regressor**.

## Linear Regression
A Linear Regression model was trained on the dataset to establish a baseline for predicting song popularity. The model’s performance was assessed using Root Mean Squared Error (RMSE) and 10-fold cross-validation to evaluate its generalization ability.

#### Performance Metrics:

	•The training RMSE was 10.473, indicating a moderate level of prediction error.
	•Cross-validation results across 10 folds showed an average RMSE of 10.879, with a standard deviation of 1.595, this shows some variability in performance across different subsets of data.
	•Individual fold RMSE values ranged from 7.861 to 13.664.

#### Learning Curve Analysis:

	•The learning curve reveals that both training and validation errors stabilize as the dataset size increases, but they remain relatively high, suggesting underfitting (high bias).
	•The training error starts low but quickly increases, converging near the validation error, which indicates that adding more data alone may not significantly improve model performance.
	•The gap between the two curves is small, meaning that overfitting is not a major concern, but the model may be too simple to fully capture the underlying relationships in the data.

In [ ]:
lin_reg=LinearRegression()
# lin_reg = TransformedTargetRegressor(regressor = lin_reg, func = log_transform, inverse_func = inverse_transform)
lin_reg.fit(X_train, y_train)

y_lin_eval = lin_reg.predict(X_train) # Change to X_test

#model evaluation
lin_mse = mean_squared_error(y_lin_eval, y_train)   
lin_rmse = np.sqrt(lin_mse)                        

lin_cv_scores = cross_val_score(lin_reg, X_train, y_train,
scoring="neg_root_mean_squared_error", cv=10)


print('Cross Validation')
display_scores(lin_cv_scores, negative_result = True)


print('The RMSE of Linear Regression is: {:.3f}\n'.format(lin_rmse))

#plotting the learning curve

print('Learning Curve for Linear Regression model:\n')

train_errors = plot_learning_curve(lin_reg, X_train, y_train)

## Random Forest Regressor

A Random Forest Regressor (RFR) was trained using 300 trees, with no maximum depth restriction and bootstrap sampling enabled. The model was evaluated using Root Mean Squared Error (RMSE) and 10-fold cross-validation to assess its predictive performance and generalization ability.

#### Performance Metrics:
	•The training RMSE was 3.937, significantly lower than the validation scores, indicating strong performance on the training data.
	•Cross-validation results yielded an average RMSE of 10.744, with a standard deviation of 1.250, showing lower variability compared to linear regression.
	•Fold-wise RMSE values ranged from 8.927 to 12.577, indicating that the model is more stable.

#### Learning Curve Analysis:
	•The learning curve shows a clear separation between training and validation errors, suggesting overfitting—the model performs well on training data but struggles to generalize to unseen data.
	•The training error remains low, meaning the model captures complex patterns in the training set.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr_reg = RandomForestRegressor(
    n_estimators=450,      
    max_depth=None,
    bootstrap=True, 
    random_state=42        
)

rfr_reg.fit(X_train, y_train)

y_rfr_eval = rfr_reg.predict(X_train)
y_pred_best=rfr_reg.predict(X_test)
# --- MSE, then take sqrt for RMSE ---
rfr_mse = mean_squared_error(y_train, y_rfr_eval) 
rfr_rmse = np.sqrt(rfr_mse)

rfr_cv_scores = cross_val_score(rfr_reg, X_train, y_train,
scoring="neg_root_mean_squared_error", cv=10)

print('Cross Validation')
display_scores(rfr_cv_scores, negative_result = True)

print('The RMSE of Rfr Regression is: {:.3f}\n'.format(rfr_rmse))

#plotting the learning curve
print('Learning Curve for Rfr Regression model:\n')
plot_learning_curve(rfr_reg, X_train, y_train)

## XGBoost Regressor

An XGBoost Regressor was trained with moderate regularization (reg_lambda=2, reg_alpha=1) and hyperparameters such as max_depth=4 and n_estimators=120, wrapped in a log-transform via TransformedTargetRegressor to help linearize the target variable.

Performance Metrics:

	•Training RMSE: 3.937, indicating the model fits the training data very close.
	•Cross-Validation: The average RMSE over 10 folds is 11.145 (±1.537).
	•Fold RMSE scores range from 8.927 to 13.382, reflects moderate variability across different splits of the data.

Learning Curve Insights:

	•The training error decreases rapidly and remains low, while the validation error is notably higher.
	

In [ ]:
xgb_reg = XGBRegressor(max_depth=6, n_estimators=120, reg_lambda=2, reg_alpha=1, subsample=0.925, colsample_bytree=0.925, verbosity = 0)

xgb_reg = TransformedTargetRegressor(regressor = xgb_reg, func = log_transform, inverse_func = inverse_transform)

xgb_reg.fit(X_train, y_train)

#model prediction
y_pred = xgb_reg.predict(X_test)
y_xgb_eval = xgb_reg.predict(X_train)

xgb_mse = mean_squared_error(y_train, y_xgb_eval) 
xgb_rmse  = np.sqrt(rfr_mse)

xgb_cv_scores = cross_val_score(xgb_reg, X_train, y_train,
scoring="neg_root_mean_squared_error", cv=10)

print('Cross Validation')
display_scores(xgb_cv_scores, negative_result = True)

#print('Learning Curve for XGBoost model:\n')
#plot_learning_curve(xgb_reg, X_train, y_train)

print('The training RMSE of XGBoost Regression is: {:.3f}'.format(xgb_rmse))

In [ ]:
training_RMSE = [lin_rmse, xgb_rmse,rfr_rmse] 
training_RMSE = pd.DataFrame({'Linear Training RMSE':lin_rmse, 'XGBoost RMSE':xgb_rmse, 'RandomForest RMSE':rfr_rmse}, index = training_RMSE)

fig, ax = plt.subplots(figsize = (10,10))
sns.barplot(data = training_RMSE)
ax.set(xlabel='Model', ylabel=' Training RMSE')

*XGBoost RMSE and Random Forest RMSE have higher trainning rmse than linear model due to overfitting. Betweem these two fonts, randomforest has a lightly better performance than XGBoost.*

### Finalizing Predictions for Submission

In [ ]:


predictions = pd.DataFrame(data = y_pred, columns = ['pop'])

id = pd.DataFrame(data = data_test['Id'], columns = ['Id'])
predictions = pd.merge(id, predictions, right_index = True, left_index = True)

predictions
predictions.to_csv('spotify_regression.csv', index=False)

<a id="conclusions"></a>
## Conclusions

In this project, we explored how numeric and categorical features can be leveraged to predict the popularity of Spotify songs. By iteratively cleaning the data, engineering new features (e.g., combining duration, loudness, and danceability), and applying transformations to reduce skewness and standardize distributions, we ensured that our dataset was well-prepared for modeling.

Three different regression models—Linear Regression, Random Forest, and XGBoost—were then trained and assessed using RMSE as the key performance metric:

	1.Linear Regression provided a good baseline with relatively narrow gaps between training and validation errors.
	2.Random Forest demonstrated strong training performance and captured more complex relationships in the data but faced overfitting, as evident in the gap between training and validation RMSE.
	3.XGBoost offered similarly powerful predictive capabilities and allowed for regularization, yet it also exhibited overfitting. 

Overall, each model contributed valuable insights into the data’s underlying structure and helped identify potential approaches to improving prediction. However, the results may also be influenced by the limitations of the dataset itself, such as the relatively small sample size and potential biases in Spotify’s popularity metric, which is influenced by internal algorithms and external factors like playlist placement and marketing. A larger, more diverse dataset including more songs from various genres, user engagement metrics, or alternative sources could improve model performance and provide a more comprehensive understanding of song popularity. The Randomforest regression model that we model get to the 5th place in the competition finally.